** song id, 제목, 가수, 가사 크롤링 **

In [10]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 멜론 TOP 100 차트 페이지 URL
url = 'https://www.melon.com/chart/index.htm'

# 웹 페이지 요청
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)

# BeautifulSoup 객체 생성
soup = BeautifulSoup(response.text, 'html.parser')

# 곡 ID, 제목, 아티스트 추출
song_ids = []
song_titles = []
song_artists = []

for song_tag in soup.find_all('tr', class_='lst50') + soup.find_all('tr', class_='lst100'):
    song_id = song_tag.get('data-song-no')
    if song_id:
        title_tag = song_tag.find('div', class_='rank01').find('a')
        artist_tag = song_tag.find('div', class_='rank02').find('a')
        song_ids.append(song_id)
        song_titles.append(title_tag.text)
        song_artists.append(artist_tag.text)

# 곡 가사 추출 함수
def get_lyrics(song_id):
    song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
    song_response = requests.get(song_url, headers=headers)
    song_soup = BeautifulSoup(song_response.text, 'html.parser')
    lyrics_tag = song_soup.find('div', class_='lyric')
    if lyrics_tag:
        return lyrics_tag.text.strip()
    else:
        return ""

# 가사 추출 및 저장
lyrics_list = []
for song_id in song_ids:
    lyrics = get_lyrics(song_id)
    lyrics_list.append(lyrics)
    time.sleep(1)  # 웹사이트에 부하를 주지 않기 위해 1초 대기

# 데이터프레임 생성
data = {'Song ID': song_ids, 'Title': song_titles, 'Artist': song_artists, 'Lyrics': lyrics_list}
df = pd.DataFrame(data)

# CSV 파일로 저장
df.to_csv('melon_top_100_with_lyrics.csv', index=False, encoding='utf-8-sig')

print("CSV 파일이 저장되었습니다.")


CSV 파일이 저장되었습니다.


불용어 추가 버전(미완성)

In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 불용어 텍스트 파일에서 불용어 리스트 읽기
def load_stopwords(filepath):
    with open(filepath, 'r', encoding='utf-8') as file:
        stopwords = file.read().splitlines()
    return stopwords

# 불용어 리스트 로드
stopwords = load_stopwords('stopwords.txt')

# 멜론 TOP 100 차트 페이지 URL
url = 'https://www.melon.com/chart/index.htm'

# 웹 페이지 요청
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)

# BeautifulSoup 객체 생성
soup = BeautifulSoup(response.text, 'html.parser')

# 곡 ID, 제목, 아티스트 추출
song_ids = []
song_titles = []
song_artists = []

for song_tag in soup.find_all('tr', class_='lst50') + soup.find_all('tr', class_='lst100'):
    song_id = song_tag.get('data-song-no')
    if song_id:
        title_tag = song_tag.find('div', class_='rank01').find('a')
        artist_tag = song_tag.find('div', class_='rank02').find('a')
        song_ids.append(song_id)
        song_titles.append(title_tag.text)
        song_artists.append(artist_tag.text)

# 곡 가사 추출 함수
def get_lyrics(song_id):
    song_url = f'https://www.melon.com/song/detail.htm?songId={song_id}'
    song_response = requests.get(song_url, headers=headers)
    song_soup = BeautifulSoup(song_response.text, 'html.parser')
    lyrics_tag = song_soup.find('div', class_='lyric')
    if lyrics_tag:
        return lyrics_tag.text.strip()
    else:
        return ""

# 가사에서 불용어 제거 함수
def remove_stopwords(text, stopwords):
    words = text.split()
    meaningful_words = [word for word in words if word not in stopwords]
    return ' '.join(meaningful_words)

# 가사 추출 및 불용어 제거
lyrics_list = []
for song_id in song_ids:
    lyrics = get_lyrics(song_id)
    clean_lyrics = remove_stopwords(lyrics, stopwords)
    lyrics_list.append(clean_lyrics)
    time.sleep(1)  # 웹사이트에 부하를 주지 않기 위해 1초 대기

# 데이터프레임 생성
data = {'Song ID': song_ids, 'Title': song_titles, 'Artist': song_artists, 'Lyrics': lyrics_list}
df = pd.DataFrame(data)

# CSV 파일로 저장
df.to_csv('melon_top_100_with_clean_lyrics.csv', index=False, encoding='utf-8-sig')

print("CSV 파일이 저장되었습니다.")


CSV 파일이 저장되었습니다.
